In [11]:
import uproot
import pandas as pd 
import numpy as np 
from particle_properties import particle_properties
from jet_properties import jet_properties



data  = uproot.open('./tag_1_delphes_events.root')['Delphes']
#data.show()

particle = particle_properties(data)

Length = len(particle.event)
test_length = 10
#df0 = particle.dataframelize(0)
#df1 = particle.dataframelize(1)
#df2 = particle.dataframelize(2)

def shift_particle_tracing(dataset, PID_d, idx):
    if (dataset.iloc[idx,6] == PID_d):
        return dataset.iloc[idx,4]

def particle_tracing(dataset, PID, STATUS):

    for i in range(len(dataset)):
        if(dataset.iloc[i,1] == STATUS and dataset.iloc[i,6] == PID ): 
            daughter_index = int(dataset.iloc[i,0])
    if( dataset.iloc[daughter_index,6] == PID ):
        shifted_particle_index = dataset.iloc[daughter_index, 4]


    while dataset.iloc[shifted_particle_index,6] == PID:
            init_shifted_particle_index = shifted_particle_index
            shifted_particle_index = shift_particle_tracing(dataset, PID, init_shifted_particle_index)       

    dauthter_idx_1 = dataset.iloc[init_shifted_particle_index, 4]
    daughter_pid_1 = dataset.iloc[dauthter_idx_1, 6]

    dauthter_idx_2 = dataset.iloc[init_shifted_particle_index, 5]
    daughter_pid_2 = dataset.iloc[dauthter_idx_2, 6]

    return init_shifted_particle_index, dauthter_idx_1, daughter_pid_1, dauthter_idx_2, daughter_pid_2

PID_W_plus = 24 
PID_W_minus = -24
PID_DOWN = 1
PID_DOWN_VAR = -1
PID_UP = 2
PID_UP_BAR = -2
PID_STRANGE = 3
PID_STRANGE_BAR = -3
PID_CHARM = 4
PID_CHARM_BAR = -4
PID_BOTTOM = 5
PID_BOTTOM_BAR = -5
PID_TOP = 6
PID_TOP_BAR = -6

top_idx = np.zeros(len(particle.event))
top_daughter_idx_1 = np.zeros(len(particle.event))
top_daughter_pid_1 = np.zeros(len(particle.event))
top_daughter_idx_2 = np.zeros(len(particle.event))
top_daughter_pid_2 = np.zeros(len(particle.event))

top_bar_idx = np.zeros(len(particle.event))
top_bar_daughter_idx_1 = np.zeros(len(particle.event))
top_bar_daughter_pid_1 = np.zeros(len(particle.event))
top_bar_daughter_idx_2 = np.zeros(len(particle.event))
top_bar_daughter_pid_2 = np.zeros(len(particle.event))

quark_idx_1 = np.zeros(len(particle.event))

quark_idx_2 = np.zeros(len(particle.event))

quark_idx_3 = np.zeros(len(particle.event))

quark_idx_4 = np.zeros(len(particle.event))

quark_idx_5 = np.zeros(len(particle.event))

quark_idx_6 = np.zeros(len(particle.event))




for i in range(0,10):
    top_idx[i], top_daughter_idx_1[i], top_daughter_pid_1[i], top_daughter_idx_2[i], top_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP, 22)
    top_bar_idx[i], top_bar_daughter_idx_1[i], top_bar_daughter_pid_1[i], top_bar_daughter_idx_2[i], top_bar_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP_BAR, 22)




def quark_finder(dataset, mother_idx):

    def switcher(input_1, input_2, m_id):
        if m_id == 6:
            if dataset.iloc[int(input_1),6] == 24 :
                print(input_1, input_2)
                return  input_1, input_2
            else :
                print(input_1, input_2)
                return  input_2, input_1
        else :
            if dataset.iloc[int(input_1),6]  == -24 :
                print(input_1, input_2)
                return  input_1, input_2
            else :
                print(input_1, input_2)
                return  input_2, input_1

    print("Mother index: {0}".format(mother_idx))

    mother_pid = dataset.iloc[int(mother_idx), 6]

    daughter_idx_1 = dataset.iloc[int(mother_idx), 4]
    daughter_idx_2 = dataset.iloc[int(mother_idx), 5]

    branch_daughter_W_idx, daught_b_idx = switcher(daughter_idx_1, daughter_idx_2,  mother_pid)

    #status = dataset.iloc[int(branch_daughter_W_idx),1]
    
    
    init_mother_idx = branch_daughter_W_idx
    daughter_idx_1_1, daughter_idx_1_2 = dataset.iloc[int(branch_daughter_W_idx), 4], dataset.iloc[int(branch_daughter_W_idx), 5]
    
    status = dataset.iloc[int(daughter_idx_1_1),1]
    print("Daughter's of t/t~'s status code: {0}".format(status))
    print(dataset.iloc[int(daughter_idx_1_1), 1])
    #find daughter of daughter 1
    while status != 23 :    
        daughter_idx_1_1 = dataset.iloc[int(init_mother_idx), 4]
        daughter_idx_1_2 = dataset.iloc[int(init_mother_idx), 5]
        status = dataset.iloc[int(daughter_idx_1_1), 1]
        print("2nd stage daughter status code: {0}, PID: {1}".format(status, dataset.iloc[int(daughter_idx_1), 6]))
        init_mother_idx = daughter_idx_1_1
    
    
    
    daughter_pid_1_1 = dataset.iloc[daughter_idx_1_1, 6]
    daughter_pid_1_2 = dataset.iloc[daughter_idx_1_2, 6]
    
    print("Daughter 1 index: {0}, daughter 2 index: {1}".format(daughter_idx_1_1, daughter_idx_1_2))
    print("Daughter 1 PID: {0}, daughter 2 PID: {1}".format(daughter_pid_1_1, daughter_pid_1_2))


    print( int(daught_b_idx), int(daughter_idx_1_1), int(daughter_idx_1_2))
    return int(daught_b_idx), int(daughter_idx_1_1), int(daughter_idx_1_2)







1008 1008 1008


In [12]:
for i in range(0,10):
    quark_idx_1[i], quark_idx_2[i], quark_idx_3[i] = quark_finder(particle.dataframelize(i), top_daughter_idx_1[i])
    quark_idx_4[i], quark_idx_5[i], quark_idx_6[i] = quark_finder(particle.dataframelize(i), top_bar_daughter_idx_1[i])

Mother index: 270.0
274 274
Daughter's of t/t~'s status code: 23
23
Daughter 1 index: 301, daughter 2 index: 302
Daughter 1 PID: 2, daughter 2 PID: -1
274 301 302
Mother index: 250.0
254 254
Daughter's of t/t~'s status code: 23
23
Daughter 1 index: 278, daughter 2 index: 279
Daughter 1 PID: 3, daughter 2 PID: -4
254 278 279
Mother index: 358.0
362 362
Daughter's of t/t~'s status code: 23
23
Daughter 1 index: 395, daughter 2 index: 396
Daughter 1 PID: 2, daughter 2 PID: -1
362 395 396
Mother index: 387.0
391 391
Daughter's of t/t~'s status code: 23
23
Daughter 1 index: 421, daughter 2 index: 422
Daughter 1 PID: 1, daughter 2 PID: -2
391 421 422
Mother index: 672.0
676 676
Daughter's of t/t~'s status code: 23
23
Daughter 1 index: 685, daughter 2 index: 686
Daughter 1 PID: 4, daughter 2 PID: -3
676 685 686
Mother index: 683.0
717 718
Daughter's of t/t~'s status code: 51
51
2nd stage daughter status code: 51, PID: 3
2nd stage daughter status code: 52, PID: 3
2nd stage daughter status code:

KeyboardInterrupt: 

In [ ]:
df9 = particle.dataframelize(9)
df9.iloc[841,:]

In [6]:
df9.iloc[855,:]

Index         855.000000
Status         23.000000
Mother_1      841.000000
Mother_2       -1.000000
Daughter_1    858.000000
Daughter_2    858.000000
PID            -4.000000
PT             10.439919
Eta            -1.188499
Phi             1.433495
Mass            1.500000
Name: 855, dtype: float64

In [34]:
df9.iloc[854,:]

Index         854.000000
Status         23.000000
Mother_1      841.000000
Mother_2       -1.000000
Daughter_1    856.000000
Daughter_2    857.000000
PID             3.000000
PT            148.473068
Eta            -0.016396
Phi            -0.437556
Mass            0.000000
Name: 854, dtype: float64

In [19]:
df9.iloc[856,:]

Index         856.000000
Status         51.000000
Mother_1      854.000000
Mother_2       -1.000000
Daughter_1    966.000000
Daughter_2    966.000000
PID             3.000000
PT             43.345860
Eta            -0.065489
Phi            -0.440145
Mass            0.000000
Name: 856, dtype: float64

In [32]:
df9.iloc[857,:]

Index         857.000000
Status         51.000000
Mother_1      854.000000
Mother_2       -1.000000
Daughter_1    859.000000
Daughter_2    860.000000
PID            21.000000
PT            105.116669
Eta             0.003365
Phi            -0.436168
Mass            0.000000
Name: 857, dtype: float64